In [80]:
import json
import yaml
import time
import pandas as pd
import numpy as np
import datetime
import os
import sys
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from openai import OpenAI
from dateutil import parser
import json
import re
from collections import Counter
from tqdm import tqdm
import pymssql
from threading import Thread
import functools
import difflib

# GPT API key
with open('openai_apikey.txt', 'r') as file:
    apikey = file.read()
os.environ["OPENAI_API_KEY"] = apikey

def calc_metrics(ground_truth, predictions):
    ground_truth_counter = Counter(ground_truth)
    predictions_counter = Counter(predictions)

    true_positives = sum((ground_truth_counter & predictions_counter).values())
    false_positives = sum((predictions_counter - ground_truth_counter).values())
    false_negatives = sum((ground_truth_counter - predictions_counter).values())

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

def evaluate(masked, generated):
    """ 
    Input: 
        - masked (str): Ground_truth text
        - generated(str): Text to be evaluated

    Output:
        - Precision, Recall and F1 (float)
    """
    ground_truth = re.findall(r'\[\*\*(.*?)\*\*\]', masked)
    predictions = re.findall(r'\[\*\*(.*?)\*\*\]', generated)
    #print(ground_truth)
    #print(predictions)
    
    return calc_metrics(ground_truth, predictions)

def levenshtein_distance(s1, s2, show_progress=True):
    """
    Calcula la distancia de Levenshtein entre dos cadenas.

    La distancia de Levenshtein es el número mínimo de operaciones de edición 
    (inserción, eliminación o sustitución de un carácter) necesarias para 
    transformar una cadena en otra.

    Parámetros:
        s1 (str): Primera cadena
        s2 (str): Segunda cadena
        show_progress (bool): Si es True, muestra una barra de progreso. 
                              Por defecto es False.
    Retorna:
        int: La distancia de Levenshtein entre s1 y s2
    """
    # Usar tqdm solo si show_progress es True
    iterable = tqdm(s1) if show_progress else s1

    if len(s1) < len(s2):
        s1, s2 = s2, s1
    if len(s2) == 0:
        return len(s1)

    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(iterable):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def replace_special_characters(text):
    # Define the pattern to match special characters including . / and -
    pattern = r'[!@#$%^&*()_+={}\[\]:;"\'<>,?\\|`~./-]'
    
    # Replace the matched characters with a space
    result = re.sub(pattern, ' ', text)
    
    return result



def find_indices(text, word):
    """Find all indices of word in text"""
    return [m.start() for m in re.finditer(re.escape(word), text)]


def replace_nth_occurrence(text, word, n):
    """Replace the nth occurrence of word in text with [**word**]"""
    indices = find_indices(text, word)
    if n < len(indices):
        start_index = indices[n]
        end_index = start_index + len(word)
        text = text[:start_index] + '[**' + word + '**]' + text[end_index:]
    return text


def sort_by_length_descending(str_list):
    return sorted(str_list, key=len, reverse=True)

def process_text(text):
    result = []
    stack = []
    i = 0
    n = len(text)

    while i < n:
        if text[i:i+3] == '[**':
            if not stack:
                result.append('[**')
            stack.append('[**')
            i += 3
        elif text[i:i+3] == '**]':
            if stack:
                stack.pop()
            if not stack:
                result.append('**]')
            i += 3
        else:
            result.append(text[i])
            i += 1

    return ''.join(result)

def prediction_process(Prediction, Replaced):
    # Extract all words enclosed in [** **] in the Prediction text
    enclosed_words = re.findall(r'\[\*\*(.*?)\*\*\]', Prediction)
    enclosed_words = list(set(enclosed_words))
    enclosed_words = sort_by_length_descending(enclosed_words)
    
    # Create a copy of Replaced text to apply changes
    updated_replaced = Replaced

    for word in enclosed_words:
        # Find all indices of the word in the Prediction text
        prediction_indices = find_indices(Prediction, '[**' + word + '**]')
        
        # Replace the nth occurrence of word in Replaced text based on the indices in Prediction
        for idx in range(len(prediction_indices)):
            updated_replaced = replace_nth_occurrence(updated_replaced, word, idx)
            
    updated_replaced=process_text(updated_replaced)

    return updated_replaced


def partial_score(X, Y):
    if X in Y or Y in X:
        x_words = X.split()
        y_words = Y.split()

        matches = sum(1 for word in y_words if word in x_words)

        return matches / len(x_words) if len(x_words) > 0 else 0
    else:
        return 0


def evaluate2(masked,generated):
    masked=masked.replace('\n','')
    generated=generated.replace('\n','')

    ground_truth_matches = re.finditer(r'\[\*\*(.*?)\*\*\]', masked)
    ground_truth_positions = {}
    cnt=0
    for match in ground_truth_matches:
        start = match.start(1)-(cnt*2+1)*3  # start of the group (excluding [**)
        end = match.end(1)-(cnt*2+1)*3
        cnt+=1# end of the group (excluding **])
        ground_truth_positions[(start, end)] = replace_special_characters(match.group(1))

    predictions_matches = re.finditer(r'\[\*\*(.*?)\*\*\]', generated)
    predictions_positions = {}
    cnt=0
    for match in predictions_matches:
        start = match.start(1)-(cnt*2+1)*3  # start of the group (excluding [**)
        end = match.end(1)-(cnt*2+1)*3
        cnt+=1# end of the group (excluding **])
        predictions_positions[(start, end)] = replace_special_characters(match.group(1))

    totalwordcnt_ground_truth = len(ground_truth_positions)
    score_total=0
    for pos_g in ground_truth_positions:
        for pos_p in predictions_positions:
            if (pos_p[0]<=pos_g[0] and pos_p[1]>=pos_g[1]) or (pos_p[0]>=pos_g[0] and pos_p[1]<=pos_g[1]):
                score_temp = partial_score(ground_truth_positions[pos_g],predictions_positions[pos_p])
                score_total += score_temp

    score_total = score_total/totalwordcnt_ground_truth
    recall = score_total

    totalwordcnt_predictions = len(predictions_positions)
    score_total=0
    for pos_p in predictions_positions:
        for pos_g in ground_truth_positions:
            if (pos_g[0]<=pos_p[0] and pos_g[1]>=pos_p[1]) or (pos_g[0]>=pos_p[0] and pos_g[1]<=pos_p[1]):
                score_temp = partial_score(predictions_positions[pos_p],ground_truth_positions[pos_g])
                score_total += score_temp

    score_total = score_total/totalwordcnt_predictions
    precision = score_total
    
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

#ARMEN_base_path='CARMEN-I_v1.01b 2/CARMEN-I_v1.01b/txt/'
CARMEN_base_path='data/data/processed/'

txtlist=os.listdir(CARMEN_base_path+'masked/')

# This is for connecting to the MSSQL server via pymssql. Adjust it according to your configuration.
conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')

# This is the table name that you will create in the MSSQL server to save the GPT responses.
# Adjust it according to your preferences.
newtablename='20240719_test_Prediction5'

In [2]:
len(txtlist)

1539

In [3]:
df=pd.DataFrame(columns=['txtname','Replaced','Masked'])
#for i in range(10): 
for i in range(len(txtlist)): 
    with open(CARMEN_base_path+'txt/'+txtlist[i], 'r', encoding='utf-8') as file:
        Replaced = file.read()
        if Replaced[0]=='\n':
            Replaced=Replaced[1:]
        if Replaced[-1]=='\n':
            Replaced=Replaced[:-1]
    with open(CARMEN_base_path+'masked/'+txtlist[i], 'r', encoding='utf-8') as file:
        Masked = file.read()
        if Masked[0]=='\n':
            Masked=Masked[1:]
        if Masked[-1]=='\n':
            Masked=Masked[:-1]
    if len(Masked.replace('**]','').replace('[**',''))!=len(Replaced):
        print(i)
    else:
        templist=[]
        templist.append(txtlist[i])
        templist.append(Replaced) 
        templist.append(Masked) 
        df.loc[len(df)]=templist
print('length',len(df))

0
556
557
559
560
1045
1057
1163
1356
length 1530


In [4]:
for newtablename in [newtablename]:
    sql_createtable="CREATE TABLE [" + newtablename +"""] 
    (
        txtname    NVARCHAR(max),
        txt   NVARCHAR(max),
    )

    """
    conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')
    with conn:
        with conn.cursor() as cur:
            cur.execute(sql_createtable)
            conn.commit()

In [5]:
def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

            
@timeout(0.1)
def SQL_Prediction(Replaced, txtname, system_content, model_ver, numbers, temperature_setting):
    
    client = OpenAI()
    response = client.chat.completions.create(
        model=model_ver,
        n=numbers,
        temperature=temperature_setting,
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": [
                {"type": "text", "text": Replaced},
            ]}
        ],
    )
    
    Prediction=response.choices[0].message.content

    conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')
    sql = "INSERT INTO [" + newtablename+"] (txtname, txt) VALUES (%s, %s)"

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql, (txtname, Prediction))
            conn.commit()

system_content2='''You are an anonymization tool in identifying attributes in texts that can identify or quasi-identify a user.
Return only the original text with the identification and labeling of the patient's personal information by adding it between [** and **].
Following are attributes that you must anonymize.

- Names
Example:
En seguimiento por Hematología Centro Médico Aspasia (Dra. Valvanera).   ->
En seguimiento por Hematología [**Centro Médico Aspasia**] (Dra. [**Valvanera**]).

- Ages
Example: 
Varón de 41 años.   ->
[**Varón**] de [**41 años**].

- Sexes
Example: 
Varón de 41 años.   ->
[**Varón**] de [**41 años**].

- Professions
Example: 
Trabaja como profesor.   ->
Trabaja como [**profesor**].

- Relatives
Example: 
Vive con suegro y 2 yernos.   ->
Vive con [**suegro**] y 2 [**yernos**].

- Dates
Example: 
ha estado viviendo en el Centro desde septiembre de 2008.   ->
ha estado viviendo en el [**Centro**] desde [**septiembre de 2008**].

- Phone numbers
Example: 
contactando con el siguiente número de teléfono +50 88 078 68 49.   ->
contactando con el siguiente número de teléfono [**+50 88 078 68 49**].

- Identification numbers
Example:
El paciente otorga su consentimiento informado para participar en el estudio del protocolo WYX/8408/5545.   ->
El paciente otorga su consentimiento informado para participar en el estudio del protocolo [**WYX/8408/5545.**]

- Institutions, hospitals, health centers, etc
Example: 
En seguimiento por Hematología Centro Médico Aspasia (Dra. Valvanera).   ->
En seguimiento por Hematología [**Centro Médico Aspasia**] (Dra. [**Valvanera**]).
Example:
Control en Centro Salud Mental Reyes Católicos.   ->
Control en [**Centro Salud Mental Reyes Católicos**].

- Countries, territories, streets, etc
Example:
nacido en la República Italiana.   ->
nacido en la [**República Italiana**].
Example:
ha estado viviendo en el Centro desde septiembre de 2008.   ->
ha estado viviendo en el [**Centro**] desde [**septiembre de 2008**].
Example:
la dirección es Calle de Victor Hugo 39.   ->
la dirección es [**Calle de Victor Hugo 39**].

- Website URLs
participar a través del siguiente enlace: https://www.donarsang.gencat.cat/covid19.   ->
participar a través del siguiente enlace: [**https://www.donarsang.gencat.cat/covid19**].

- Other sensitive information such as races, ethnicities, sexual orientation, dietary preferences, etc
Example:
raça blanca   ->
[**raça blanca**]
Example:
Hsh
[**Hsh**]
Example:
Vegetarià
[**Vegetarià**]

Do not comment anything else.
Besides the anonymized attributes, provide the rest of the text exactly the same, including special characters and \n symbols.
Do not correct any typos or spacing errors at your discretion.
For example, if the time is written as 31/12/2000-0 9:20:00 with incorrect spacing, do not return it corrected as 31/12/2000-09:20:00.
Also, for example, if FLUTICASONA + AZELA STINA4 is written with incorrect spacing, do not return it corrected as FLUTICASONA + AZELASTINA 4.
Only focus on the anonymization tasks I have specified, and ignore any typos or spacing errors
'''

In [6]:
sample_size=300

In [7]:
for i in range(sample_size):
    txtname=df['txtname'][i]
    Replaced=df['Replaced'][i]
    SQL_Prediction(Replaced, txtname, system_content2, 'gpt-4o', 1, 1.0)

In [10]:
conn = pymssql.connect(host=r"(local)", database='Ddrive5', charset='utf8')
sql_statement="select * from ["+ newtablename + "]"
df_SQL_Prediction = pd.read_sql(sql=sql_statement, con=conn)

C:\Users\Desktop\AppData\Local\Temp\ipykernel_74740\3136040221.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL_Prediction = pd.read_sql(sql=sql_statement, con=conn)


In [12]:
while True:
    if len(df_SQL_Prediction)>=sample_size:
        break
    
    for i in range(sample_size):
        if df['txtname'][i] not in list(df_SQL_Prediction['txtname']):
            txtname=df['txtname'][i]
            Replaced=df['Replaced'][i]
            SQL_Prediction(Replaced, txtname, system_content2, 'gpt-4o', 1, 1.0)
            print('inserted')
    time.sleep(60)
    print('slept')
    cnt=0
    for i in range(sample_size):
        if df['txtname'][i] not in list(df_SQL_Prediction['txtname']):
            cnt+=1
    if cnt==0:
        break

In [81]:
df2=pd.merge(df,df_SQL_Prediction,left_on='txtname',right_on='txtname',how='inner')
df2.columns=['txtname','Replaced','Masked','Prediction']
df2['Prediction_processed'] = df2.apply(lambda row: prediction_process(row['Prediction'], row['Replaced']), axis=1)

In [82]:
precision=[]
recall=[]
f1=[]
for i in range(len(df2)):
    cal_met = evaluate2(df2['Masked'][i], df2['Prediction_processed'][i])
    precision.append(cal_met[0])
    recall.append(cal_met[1])
    f1.append(cal_met[2])

df2['precision']=precision
df2['recall']=recall
df2['f1']=f1

In [83]:
df2

,txtname,Replaced,Masked,Prediction,Prediction_processed,precision,recall,f1
0,CARMEN-I_CC_2.txt,Realizo llamada telefónica. Refiere buen desca...,Realizo llamada telefónica. Refiere buen desca...,Realizo llamada telefónica. Refiere buen desca...,Realizo llamada telefónica. Refiere buen desca...,0.952381,1.000000,0.975610
1,CARMEN-I_CC_3.txt,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,0.333333,0.333333,0.333333
2,CARMEN-I_CC_4.txt,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,Primera llamada HDOM COVID-19 POSITIVO\n.\nTra...,0.863636,1.000000,0.926829
3,CARMEN-I_CC_5.txt,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,Primera llamada HDOM COVID-19 POSITIVO\nTrabaj...,1.000000,0.956522,0.977778
4,CARMEN-I_IA_ANTECEDENTES_1.txt,"Varón de 79 años, sin alergias a medicamentos ...","[**Varón**] de [**79 años**], sin alergias a m...","[**Varón**] de [**79 años**], sin alergias a m...","[**Varón**] de [**79 años**], sin alergias a m...",0.937500,0.937500,0.937500
...,...,...,...,...,...,...,...,...
295,CARMEN-I_IA_EXPLORACION_CLINICA_2.txt,13/07/2026-19:00:00: Frecuencia respiratoria (...,[**13/07/2026**]-19:00:00: Frecuencia respirat...,[**13/07/2026-19:00:00**]: Frecuencia respirat...,[**13/07/2026-19:00:00**]: Frecuencia respirat...,0.464286,1.000000,0.634146
296,CARMEN-I_IA_EXPLORACION_CLINICA_20.txt,"19/7/2016-23:00:00: PAS (mm Hg) 112, PAD (mm H...","[**19/7/2016**]-23:00:00: PAS (mm Hg) 112, PAD...","[**19/7/2016-23:00:00**]: PAS (mm Hg) 112, PAD...","[**19/7/2016-23:00:00**]: PAS (mm Hg) 112, PAD...",0.500000,1.166667,0.700000
297,CARMEN-I_IA_EXPLORACION_CLINICA_21.txt,28/04/2025-18:00:00: Glicemia capilar (mg/dl) ...,[**28/04/2025**]-18:00:00: Glicemia capilar (m...,[**28/04/2025-18:00:00**]: Glicemia capilar (m...,[**28/04/2025-18:00:00**]: Glicemia capilar (m...,0.500000,1.000000,0.666667
298,CARMEN-I_IA_EXPLORACION_CLINICA_22.txt,26/08/2025-09:00:00: Tª axilar (ºC) 35.7. 28/0...,[**26/08/2025**]-09:00:00: Tª axilar (ºC) 35.7...,[**26/08/2025**]-09:00:00: Tª axilar (ºC) 35.7...,[**26/08/2025**]-09:00:00: Tª axilar (ºC) 35.7...,1.000000,1.000000,1.000000


In [84]:
df2[df2['recall']<0.7]

,txtname,Replaced,Masked,Prediction,Prediction_processed,precision,recall,f1
1,CARMEN-I_CC_3.txt,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,Visita unidad del dolor. Ver informe de evoluc...,0.333333,0.333333,0.333333
49,CARMEN-I_IA_ANTECEDENTES_140.txt,"Hombre de 87 años, sin alergias conocidas a me...","[**Hombre**] de [**87 años**], sin alergias co...","[**Hombre**] de [**87 años**], sin alergias co...","[**Hombre**] de [**87 años**], sin alergias co...",1.000000,0.625000,0.769231
135,CARMEN-I_IA_ANTECEDENTES_93.txt,"57 años, ilustrador en una revista.\nANTECEDEN...","[**57 años**], [**ilustrador en una revista**]...","[**57 años**], [**ilustrador**] en una revista...","[**57 años**], [**ilustrador**] en una revista...",1.000000,0.625000,0.769231
145,CARMEN-I_IA_EVOL_101.txt,EVOLUCIÓN PSIQUIATRÍA:\nLa paciente ingresa en...,EVOLUCIÓN PSIQUIATRÍA:\nLa paciente ingresa en...,EVOLUCIÓN PSIQUIATRÍA:\n[**La paciente**] ingr...,EVOLUCIÓN PSIQUIATRÍA:\n[**La paciente**] ingr...,0.264000,0.636364,0.373183
166,CARMEN-I_IA_EVOL_122.txt,// *EVOLUCIÓN P523*\n1. INSUFICIENCIA RESPIRAT...,// *EVOLUCIÓN [**P523**]*\n1. INSUFICIENCIA RE...,// *EVOLUCIÓN P523*\n1. INSUFICIENCIA RESPIRAT...,// *EVOLUCIÓN P523*\n1. INSUFICIENCIA RESPIRAT...,0.592857,0.695652,0.640154
183,CARMEN-I_IA_EVOL_138.txt,ENFERMEDADES INFECCIOSAS\nPaciente con los ant...,ENFERMEDADES INFECCIOSAS\nPaciente con los ant...,ENFERMEDADES INFECCIOSAS\nPaciente con los ant...,ENFERMEDADES INFECCIOSAS\nPaciente con los ant...,1.000000,0.694444,0.819672
207,CARMEN-I_IA_EVOL_26.txt,EVOLUCION EN SALA COVID\n1) Neumonia bilateral...,EVOLUCION EN SALA COVID\n1) Neumonia bilateral...,EVOLUCION EN SALA COVID\n1) Neumonia bilateral...,EVOLUCION EN SALA COVID\n1) Neumonia bilateral...,1.000000,0.666667,0.800000
212,CARMEN-I_IA_EVOL_30.txt,1. BACTERIEMIA POR K. OXYTOCA SENSIBLE DE FOCO...,1. BACTERIEMIA POR K. OXYTOCA SENSIBLE DE FOCO...,1. BACTERIEMIA POR K. OXYTOCA SENSIBLE DE FOCO...,1. BACTERIEMIA POR K. OXYTOCA SENSIBLE DE FOCO...,0.250000,0.666667,0.363636
280,CARMEN-I_IA_EVOL_96.txt,*EVOLUCIÓN:\n1. SCASEST KK1. FEVI 50%.\n2. DIL...,*EVOLUCIÓN:\n1. SCASEST KK1. FEVI 50%.\n2. DIL...,*EVOLUCIÓN:\n1. SCASEST KK1. FEVI 50%.\n2. DIL...,*EVOLUCIÓN:\n1. SCASEST KK1. FEVI 50%.\n2. DIL...,0.600000,0.600000,0.600000
291,CARMEN-I_IA_EXPLORACION_CLINICA_16.txt,11/03/2022-18:56:00: Frecuencia respiratoria (...,[**11/03/2022**]-18:56:00: Frecuencia respirat...,[**11/03/2022-18:56:00**]: Frecuencia respirat...,[**11/03/2022-18:56:00**]: Frecuencia respirat...,0.500000,0.666667,0.571429


In [85]:
print('recall avg',np.mean(df2['recall']))
print('precision avg',np.mean(df2['precision']))
print('f1 avg',np.mean(df2['f1']))

recall avg 0.9384772254963335
precision avg 0.7626648869288015
f1 avg 0.8165033367050305


In [106]:
df2['masked_count'] = df2['Masked'].str.count(r'\[\*\*')
total_masked_count=np.sum(df2['masked_count'])
total_recall=0
total_precision=0
total_f1=0
for i in range(len(df2)):
    total_recall+=df2['recall'][i]*df2['masked_count'][i]
    total_precision+=df2['precision'][i]*df2['masked_count'][i]
    total_f1+=df2['f1'][i]*df2['masked_count'][i]
recall_weighted_avg = total_recall/total_masked_count
precision_weighted_avg = total_precision/total_masked_count
f1_weighted_avg = total_f1/total_masked_count
print('recall weighted avg',recall_weighted_avg)
print('precision weighted avg',precision_weighted_avg)
print('f1 weightedavg',f1_weighted_avg)

recall weighted avg 0.937066073954334
precision weighted avg 0.818720531739647
f1 weightedavg 0.8597375483805075


In [108]:
df2.to_csv('20240719_CARMEN_Results.csv',index=False,encoding='utf-8')

# Viewing examples

In [95]:
a=49

In [96]:
print(df2['Replaced'][a])

Hombre de 87 años, sin alergias conocidas a medicamentos. No fumador, bebedor moderado ( 1-2 UBE/día). Reside en residencia de ancianos.
*ANTECEDENTES PERSONALES
1.- DISLIPEMIA sin tratamiento médico
2.- HIPERTENSIÓN ARTERIAL en tratamiento con inhibidores de la ECA.
3.- CARDIOPATIA ISQUEMICA Debut 1997 IAM anterolateral Killip I. rTPA sin reperfusión por lo que se realiza ACTP de rescate que muestra lesión en DA pm implantándose stent con buen resultado angiográfico sin complicaciones.1999: necrosis anteroapical, KT lesión severa a nivel de tronco común. PMC a DA, PAC a OM con buena evolución posterior
4.- INSUFICIENCIA CARDIACA. FEVI 35%.
5.- FIBRILACIÓN AURICULAR. En tratamiento anticoagulante, sin tratamiento frenador. FC habitual alrededor de 70 lpm.
6.- DEMENCIA MIXTA. En seguimiento por Neurología de Hospital Cruz Roja.
7.- INFARTO RENAL en contexto de enfermedad vascular renal en enero de 2010, por lo que inició tratamiento con sintrom.
8.- ENFERMEDAD VASCULAR DE ARTERIA MESENT

In [97]:
print(df2['Masked'][a])

[**Hombre**] de [**87 años**], sin alergias conocidas a medicamentos. No fumador, bebedor moderado ( 1-2 UBE/día). Reside en [**residencia de ancianos**].
*ANTECEDENTES PERSONALES
1.- DISLIPEMIA sin tratamiento médico
2.- HIPERTENSIÓN ARTERIAL en tratamiento con inhibidores de la ECA.
3.- CARDIOPATIA ISQUEMICA Debut [**1997**] IAM anterolateral Killip I. rTPA sin reperfusión por lo que se realiza ACTP de rescate que muestra lesión en DA pm implantándose stent con buen resultado angiográfico sin complicaciones.[**1999**]: necrosis anteroapical, KT lesión severa a nivel de tronco común. PMC a DA, PAC a OM con buena evolución posterior
4.- INSUFICIENCIA CARDIACA. FEVI 35%.
5.- FIBRILACIÓN AURICULAR. En tratamiento anticoagulante, sin tratamiento frenador. FC habitual alrededor de 70 lpm.
6.- DEMENCIA MIXTA. En seguimiento por Neurología de [**Hospital Cruz Roja**].
7.- INFARTO RENAL en contexto de enfermedad vascular renal en [**enero de 2010**], por lo que inició tratamiento con sintrom.

In [98]:
print(df2['Prediction'][a])

[**Hombre**] de [**87 años**], sin alergias conocidas a medicamentos. No fumador, bebedor moderado ( 1-2 UBE/día). Reside en residencia de ancianos.
*ANTECEDENTES PERSONALES
1.- DISLIPEMIA sin tratamiento médico
2.- HIPERTENSIÓN ARTERIAL en tratamiento con inhibidores de la ECA.
3.- CARDIOPATIA ISQUEMICA Debut 1997 IAM anterolateral Killip I. rTPA sin reperfusión por lo que se realiza ACTP de rescate que muestra lesión en DA pm implantándose stent con buen resultado angiográfico sin complicaciones.1999: necrosis anteroapical, KT lesión severa a nivel de tronco común. PMC a DA, PAC a OM con buena evolución posterior
4.- INSUFICIENCIA CARDIACA. FEVI 35%.
5.- FIBRILACIÓN AURICULAR. En tratamiento anticoagulante, sin tratamiento frenador. FC habitual alrededor de 70 lpm.
6.- DEMENCIA MIXTA. En seguimiento por Neurología de [**Hospital Cruz Roja**].
7.- INFARTO RENAL en contexto de enfermedad vascular renal en [**enero de 2010**], por lo que inició tratamiento con sintrom.
8.- ENFERMEDAD VA

In [99]:
print(df2['Prediction_processed'][a])

[**Hombre**] de [**87 años**], sin alergias conocidas a medicamentos. No fumador, bebedor moderado ( 1-2 UBE/día). Reside en residencia de ancianos.
*ANTECEDENTES PERSONALES
1.- DISLIPEMIA sin tratamiento médico
2.- HIPERTENSIÓN ARTERIAL en tratamiento con inhibidores de la ECA.
3.- CARDIOPATIA ISQUEMICA Debut 1997 IAM anterolateral Killip I. rTPA sin reperfusión por lo que se realiza ACTP de rescate que muestra lesión en DA pm implantándose stent con buen resultado angiográfico sin complicaciones.1999: necrosis anteroapical, KT lesión severa a nivel de tronco común. PMC a DA, PAC a OM con buena evolución posterior
4.- INSUFICIENCIA CARDIACA. FEVI 35%.
5.- FIBRILACIÓN AURICULAR. En tratamiento anticoagulante, sin tratamiento frenador. FC habitual alrededor de 70 lpm.
6.- DEMENCIA MIXTA. En seguimiento por Neurología de [**Hospital Cruz Roja**].
7.- INFARTO RENAL en contexto de enfermedad vascular renal en [**enero de 2010**], por lo que inició tratamiento con sintrom.
8.- ENFERMEDAD VA

In [100]:
masked=df2['Masked'][a]
generated=df2['Prediction_processed'][a]

In [101]:
masked=masked.replace('\n','')
generated=generated.replace('\n','')

ground_truth_matches = re.finditer(r'\[\*\*(.*?)\*\*\]', masked)
ground_truth_positions = {}
cnt=0
for match in ground_truth_matches:
    start = match.start(1)-(cnt*2+1)*3  # start of the group (excluding [**)
    end = match.end(1)-(cnt*2+1)*3
    cnt+=1# end of the group (excluding **])
    ground_truth_positions[(start, end)] = replace_special_characters(match.group(1))

predictions_matches = re.finditer(r'\[\*\*(.*?)\*\*\]', generated)
predictions_positions = {}
cnt=0
for match in predictions_matches:
    start = match.start(1)-(cnt*2+1)*3  # start of the group (excluding [**)
    end = match.end(1)-(cnt*2+1)*3
    cnt+=1# end of the group (excluding **])
    predictions_positions[(start, end)] = replace_special_characters(match.group(1))

totalwordcnt_ground_truth = len(ground_truth_positions)
score_total=0
for pos_g in ground_truth_positions:
    for pos_p in predictions_positions:
        if (pos_p[0]<=pos_g[0] and pos_p[1]>=pos_g[1]) or (pos_p[0]>=pos_g[0] and pos_p[1]<=pos_g[1]):
            score_temp = partial_score(ground_truth_positions[pos_g],predictions_positions[pos_p])
            score_total += score_temp
            print(score_temp,ground_truth_positions[pos_g])

score_total = score_total/totalwordcnt_ground_truth
recall = score_total

totalwordcnt_predictions = len(predictions_positions)
score_total=0
for pos_p in predictions_positions:
    for pos_g in ground_truth_positions:
        if (pos_g[0]<=pos_p[0] and pos_g[1]>=pos_p[1]) or (pos_g[0]>=pos_p[0] and pos_g[1]<=pos_p[1]):
            score_temp = partial_score(predictions_positions[pos_p],ground_truth_positions[pos_g])
            score_total += score_temp

score_total = score_total/totalwordcnt_predictions
precision = score_total

f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0



1.0 Hombre
1.0 87 años
1.0 Hospital Cruz Roja
1.0 enero de 2010
1.0 Porto Alegre


In [102]:
recall

0.625

In [103]:
ground_truth_positions

{(0, 6): 'Hombre',
 (10, 17): '87 años',
 (113, 135): 'residencia de ancianos',
 (296, 300): '1997',
 (487, 491): '1999',
 (812, 830): 'Hospital Cruz Roja',
 (893, 906): 'enero de 2010',
 (1086, 1098): 'Porto Alegre'}

In [104]:
predictions_positions

{(0, 6): 'Hombre',
 (10, 17): '87 años',
 (812, 830): 'Hospital Cruz Roja',
 (893, 906): 'enero de 2010',
 (1086, 1098): 'Porto Alegre'}